In [1]:
import duckdb
import pathlib as pl
import numpy as np
import pandas as pd
from IPython.display import display

# automatically locates the project root and set up relative paths to the data folder
# this makes the notebook reproducible for anyone who clones the repo
ROOT = pl.Path.cwd().parent
DATA = ROOT / "MIMIC_ED"
RAW = DATA / "raw" / "mimicel.csv"

# relative paths
RAW = pl.Path("../../MIMIC_ED/raw/mimicel.csv")
CLEAN = pl.Path("../../MIMIC_ED/cleaned/mimicel_clean.csv")

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 2000)        # Increase total display width

In [2]:
con = duckdb.connect()

df = con.execute(f"""
    SELECT *
    FROM read_csv_auto('{RAW}', HEADER=TRUE)
""").df()
df.shape

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(7568824, 31)

In [3]:
df.head()

,stay_id,subject_id,hadm_id,timestamps,activity,gender,race,arrival_transport,disposition,seq_num,icd_code,icd_version,icd_title,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint,rhythm,name,gsn,ndc,etc_rn,etccode,etcdescription,med_rn,gsn_rn
0,30000012,11714491,21562392,2126-02-14 20:22:00,Vital sign check,None,None,None,None,<NA>,None,<NA>,None,98.8,96.0,18.0,93.0,160.0,54.0,0,NaN,None,None,None,None,None,<NA>,None,None,<NA>,<NA>
1,30000012,11714491,21562392,2126-02-14 20:22:00,Enter the ED,F,WHITE,AMBULANCE,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,<NA>,None,None,<NA>,<NA>
2,30000012,11714491,21562392,2126-02-14 20:22:01,Triage in the ED,None,None,None,None,<NA>,None,<NA>,None,98.8,96.0,18.0,93.0,160.0,54.0,0,2.0,CHANGE IN MENTAL STATUS,None,None,None,None,<NA>,None,None,<NA>,<NA>
3,30000012,11714491,21562392,2126-02-14 22:21:00,Medicine reconciliation,None,None,None,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,rifaximin,066295,54868615400,1,00005844,Rifamycins and Related Derivative Antibiotics,<NA>,<NA>
4,30000012,11714491,21562392,2126-02-14 22:21:00,Medicine reconciliation,None,None,None,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,"multivitamin,tx-minerals",002510,10267073710,1,00000704,Multivitamin and Mineral Combinations,<NA>,<NA>


In [4]:
# Group by stay_id and compute summary flags
by_stay = (
    df.groupby('stay_id', dropna=True)
      .agg(
          has_hadm=('hadm_id', lambda s: s.notna().any()),
          ever_admitted=('disposition', lambda s: (s == 'ADMITTED').any())
      )
)

# 1. Stays that were ADMITTED but have NO hadm_id
admitted_no_hadm = by_stay[(by_stay['ever_admitted']) & (~by_stay['has_hadm'])]

# 2. Stays that were NEVER ADMITTED but DO have a hadm_id
not_admitted_with_hadm = by_stay[(~by_stay['ever_admitted']) & (by_stay['has_hadm'])]

print("Total unique stay_ids:", len(by_stay))
print("Admitted but missing hadm_id:", len(admitted_no_hadm))
print("Not admitted but has hadm_id:", len(not_admitted_with_hadm))

Total unique stay_ids: 425028
Admitted but missing hadm_id: 384
Not admitted but has hadm_id: 45390


In [5]:
df['activity'].value_counts().head(20)

activity
Medicine reconciliation    2953118
Medicine dispensations     1441839
Vital sign check           1423734
Discharge from the ED       900077
Enter the ED                425028
Triage in the ED            425028
Name: count, dtype: int64

In [6]:
events = df.copy()
# ensure timestamps is datetime
events["timestamps"] = pd.to_datetime(events["timestamps"])
events

,stay_id,subject_id,hadm_id,timestamps,activity,gender,race,arrival_transport,disposition,seq_num,icd_code,icd_version,icd_title,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint,rhythm,name,gsn,ndc,etc_rn,etccode,etcdescription,med_rn,gsn_rn
0,30000012,11714491,21562392,2126-02-14 20:22:00,Vital sign check,None,None,None,None,<NA>,None,<NA>,None,98.8,96.0,18.0,93.0,160.0,54.0,0,NaN,None,None,None,None,None,<NA>,None,None,<NA>,<NA>
1,30000012,11714491,21562392,2126-02-14 20:22:00,Enter the ED,F,WHITE,AMBULANCE,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,<NA>,None,None,<NA>,<NA>
2,30000012,11714491,21562392,2126-02-14 20:22:01,Triage in the ED,None,None,None,None,<NA>,None,<NA>,None,98.8,96.0,18.0,93.0,160.0,54.0,0,2.0,CHANGE IN MENTAL STATUS,None,None,None,None,<NA>,None,None,<NA>,<NA>
3,30000012,11714491,21562392,2126-02-14 22:21:00,Medicine reconciliation,None,None,None,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,rifaximin,066295,54868615400,1,00005844,Rifamycins and Related Derivative Antibiotics,<NA>,<NA>
4,30000012,11714491,21562392,2126-02-14 22:21:00,Medicine reconciliation,None,None,None,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,"multivitamin,tx-minerals",002510,10267073710,1,00000704,Multivitamin and Mineral Combinations,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7568819,39999964,16479007,29821286,2130-06-06 18:27:00,Discharge from the ED,None,None,None,HOME,2,R45851,10,Suicidal ideations,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,<NA>,None,None,<NA>,<NA>
7568820,39999965,14733226,27008423,2125-09-14 00:46:00,Enter the ED,F,BLACK/AFRICAN AMERICAN,WALK IN,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,<NA>,None,None,<NA>,<NA>
7568821,39999965,14733226,27008423,2125-09-14 00:46:01,Triage in the ED,None,None,None,None,<NA>,None,<NA>,None,97.5,65.0,16.0,100.0,132.0,77.0,0,2.0,Labor,None,None,None,None,<NA>,None,None,<NA>,<NA>
7568822,39999965,14733226,27008423,2125-09-14 21:22:50,Discharge from the ED,None,None,None,TRANSFER,1,64893,9,OTH CURR COND-ANTEPARTUM,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,<NA>,None,None,<NA>,<NA>


In [7]:
# helper function to pull values like race, gender, etc out of column instead of just taking first row entry 
def first_not_null(s: pd.Series):
    s_non_null = s.dropna()
    if len(s_non_null) == 0:
        return pd.NA
    return s_non_null.iloc[0]

## Four-Table Architecture for the ED Simulation

| Table Name       | Granularity                | Purpose / Contents                                      | Why It Matters |
|------------------|-----------------------------|----------------------------------------------------------|----------------|
| **ed_stays**     | 1 row per **ED visit**      | High-level summary: arrival, triage, depart timestamps; demographics; transport; disposition | Defines the population of stays; used for LOS, arrival modeling, routing |
| **ed_activities**| Many rows per **stay_id**   | Detailed event timeline: vitals, meds, reconciliation, etc. | Feeds the DES with activity sequences and durations |
| **ed_diagnoses** | 1 row per **ICD code** per stay | Diagnosis profile: icd_code, icd_version, icd_title | Describes medical conditions impacting flow or resource needs |
| **ed_medications** | 1 row per **medication record** per stay | Medication details: name, gsn, ndc, med_rn, etc.; may include timestamp | Supports medication-driven event timing and resource modeling |

### Summary
- **ed_stays** = stay-level summary  
- **ed_activities** = event-level timeline  
- **ed_diagnoses** = diagnosis-level detail  
- **ed_medications** = medication-level detail  

This normalization is intentional and crucial for building a clean, efficient Discrete-Event Simulation (DES).


In [8]:
arrival_time = (
    events.loc[events["activity"] == "Enter the ED"]
    .groupby("stay_id")["timestamps"]
    .min()
    .rename("arrival_time")
)

triage_time = (
    events.loc[events["activity"] == "Triage in the ED"]
    .groupby("stay_id")["timestamps"]
    .min()
    .rename("triage_time")
)

depart_time = (
    events.loc[events["activity"] == "Discharge from the ED"]
    .groupby("stay_id")["timestamps"]
    .max()
    .rename("depart_time")
)

In [9]:
stay_level = (
    events
    .groupby("stay_id")
    .agg(
        subject_id=("subject_id", "first"),
        hadm_id=("hadm_id", "first"),
        gender=("gender", first_not_null),
        race=("race", first_not_null),
        arrival_transport=("arrival_transport", first_not_null),
        disposition=("disposition", first_not_null),
        acuity=("acuity", first_not_null),
        chiefcomplaint=("chiefcomplaint", first_not_null),
    )
)

In [10]:
ed_stays = (
    stay_level
    .join([arrival_time, triage_time, depart_time])
)

In [11]:
ed_stays["ed_los_hours"] = (
    ed_stays["depart_time"] - ed_stays["arrival_time"]
).dt.total_seconds() / 3600

ed_stays["triage_delay_minutes"] = (
    ed_stays["triage_time"] - ed_stays["arrival_time"]
).dt.total_seconds() / 60

In [12]:
ed_stays = ed_stays.reset_index()

In [13]:
ed_stays.head(15)

,stay_id,subject_id,hadm_id,gender,race,arrival_transport,disposition,acuity,chiefcomplaint,arrival_time,triage_time,depart_time,ed_los_hours,triage_delay_minutes
0,30000012,11714491,21562392,F,WHITE,AMBULANCE,ADMITTED,2.0,CHANGE IN MENTAL STATUS,2126-02-14 20:22:00,2126-02-14 20:22:01,2126-02-15 01:59:00,5.616667,0.016667
1,30000017,14230614,<NA>,M,BLACK/AFRICAN AMERICAN,WALK IN,ELOPED,3.0,"ETOH, Unable to ambulate",2185-06-18 11:51:00,2185-06-18 11:51:01,2185-06-18 15:53:00,4.033333,0.016667
2,30000038,13821532,26255538,F,WHITE,AMBULANCE,ADMITTED,3.0,Cough,2152-12-07 16:37:00,2152-12-07 16:37:01,2152-12-07 19:55:00,3.300000,0.016667
3,30000039,13340997,23100190,M,WHITE,WALK IN,ADMITTED,3.0,s/p Fall,2165-10-06 11:47:00,2165-10-06 11:47:01,2165-10-06 20:18:00,8.516667,0.016667
4,30000055,19848164,<NA>,F,OTHER,WALK IN,HOME,3.0,L Ear pain,2155-07-18 17:03:00,2155-07-18 17:03:01,2155-07-18 21:12:00,4.150000,0.016667
5,30000094,19862552,<NA>,F,WHITE,AMBULANCE,HOME,2.0,N,2183-09-04 16:08:00,2183-09-04 16:08:01,2183-09-05 00:12:00,8.066667,0.016667
6,30000112,13333760,<NA>,F,HISPANIC/LATINO - PUERTO RICAN,WALK IN,HOME,3.0,Abnormal labs,2157-12-12 12:45:00,2157-12-12 12:45:01,2157-12-12 15:22:00,2.616667,0.016667
7,30000177,17937834,23831044,M,ASIAN - SOUTH EAST ASIAN,WALK IN,ADMITTED,2.0,"Abd pain, Vomiting",2143-12-27 22:50:00,2143-12-27 22:50:01,2143-12-28 03:48:00,4.966667,0.016667
8,30000202,15346940,<NA>,M,BLACK/AFRICAN AMERICAN,WALK IN,HOME,2.0,ALLERGIC REACTION,2158-08-08 10:07:00,2158-08-08 10:07:01,2158-08-08 18:23:00,8.266667,0.016667
9,30000204,11615015,25540031,M,WHITE,AMBULANCE,HOME,3.0,S/P ASSAULT,2132-10-10 06:36:00,2132-10-10 06:36:01,2132-10-10 18:45:00,12.150000,0.016667


In [14]:
# adding is_admitted in case we link this data with the ICU MIMIC dataset
# decision node at ED end could be if is_admitted: route to hosital bed/ICU, else route to home/left/other
admit_dispositions = ["ADMITTED"]
ed_stays["is_admitted"] = ed_stays["disposition"].isin(admit_dispositions)
ed_stays[["stay_id", "disposition", "is_admitted"]].head(10)

,stay_id,disposition,is_admitted
0,30000012,ADMITTED,True
1,30000017,ELOPED,False
2,30000038,ADMITTED,True
3,30000039,ADMITTED,True
4,30000055,HOME,False
5,30000094,HOME,False
6,30000112,HOME,False
7,30000177,ADMITTED,True
8,30000202,HOME,False
9,30000204,HOME,False


## Note on Triage Timestamp Accuracy in MIMIC-IV ED

While constructing `ed_stays`, we computed a variable called **`triage_delay_minutes`**, defined as the time difference between:

- **arrival_time** (first "Enter the ED"), and  
- **triage_time** (first "Triage in the ED").

During exploration, we found that *nearly every patient* in the MIMIC-IV ED event log has:

- **arrival_time = HH:MM:SS**
- **triage_time = HH:MM:SS + 1 second**

This produces a constant value of **0.016667 minutes** (i.e., 1 second) for almost the entire dataset.

### Why This Happens
This behavior is **not** a coding error. It reflects how timestamps are recorded in the underlying MIMIC ED event log:

- Many EDs log triage and registration events in batches.
- Timestamps are often rounded or defaulted to the same second.
- The dataset uses shifted dates and harmonized event logs, which further compress timing resolution.
- The “Triage in the ED” event frequently represents when the *triage note was charted*, not when the nurse actually triaged the patient.

As a result, **triage timestamps in MIMIC-IV ED do not represent real-world triage wait times.**

### Implications for the DES Model
For our simulation:
- We cannot infer real triage wait times from MIMIC.
  The data does not contain the necessary temporal resolution.
- We should keep the `triage_time` field for ordering events, but we should not use `triage_delay_minutes` for modeling actual triage delays.
- We do plan to switch to UCSD Health Data so keeping this metric is still important.

### What We *Can* Still Use from MIMIC
Despite the triage-timing limitations, MIMIC-IV ED remains extremely useful for:

- **Arrival → depart timeframes** (accurate ED length of stay)
- **Activity ordering** (the sequence of ED events is reliable)
- **Inter-activity timing for meds, vitals, procedures**
- **Medication and diagnosis timestamps**
- **Disposition (discharge, transfer, admission)**

These provide strong, real-world foundations for modeling ED flow.

### How to Handle Triage Delay in the Simulation
Since MIMIC cannot provide this:
- Use **external literature values** (typical triage waits are 5–20 minutes)
- OR sample from a **distribution** (e.g., lognormal or gamma) based on national datasets like **NHAMCS**
- OR choose a fixed **constant delay** if the model does not focus on triage bottlenecks

We will therefore keep the triage timestamps in the dataset for completeness, but treat the actual delay as **unreliable** and **not suitable for DES parameterization**.

In [15]:
# next few cells will be doing some sanity checks on the df to check for completeness
ed_stays[["stay_id", "subject_id", "arrival_time", "depart_time", "disposition"]].isna().mean()

stay_id         0.0
subject_id      0.0
arrival_time    0.0
depart_time     0.0
disposition     0.0
dtype: float64

In [16]:
# confirming timestamp ordering
(ed_stays["triage_time"] < ed_stays["arrival_time"]).sum()

np.int64(0)

In [17]:
# same here
(ed_stays["depart_time"] < ed_stays["arrival_time"]).sum()

np.int64(0)

In [18]:
# checking disposition LOS values for outliers or anything concerning, this all looks normal.
ed_stays["ed_los_hours"].describe(percentiles=[0.01,0.25,0.5,0.75,0.99])

count    425028.000000
mean          7.159524
std           6.627396
min           0.016667
1%            0.766667
25%           3.533333
50%           5.466667
75%           8.316667
99%          30.483333
max         493.069444
Name: ed_los_hours, dtype: float64

In [19]:
# checking for zero or negative LOS
(ed_stays["ed_los_hours"] <= 0).sum()

np.int64(0)

In [20]:
# checking demographic completness, I am shocked at how complete this is. Very nice.
ed_stays[["gender", "race", "acuity", "chiefcomplaint"]].isna().mean()

gender            0.000000
race              0.000000
acuity            0.016413
chiefcomplaint    0.000047
dtype: float64

In [21]:
# checking that disposition looks reasonable, it does
ed_stays["disposition"].value_counts(dropna=False)

disposition
HOME                           241626
ADMITTED                       158010
TRANSFER                         7025
LEFT WITHOUT BEING SEEN          6154
ELOPED                           5710
OTHER                            4245
LEFT AGAINST MEDICAL ADVICE      1881
EXPIRED                           377
Name: count, dtype: int64

In [22]:
# save this codle block for running code that creates a csv from ed_stays

In [23]:
# now on to building ed_activities
activities = df.copy()
activities = activities[activities["activity"].notna()].copy()
activities.head()

,stay_id,subject_id,hadm_id,timestamps,activity,gender,race,arrival_transport,disposition,seq_num,icd_code,icd_version,icd_title,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint,rhythm,name,gsn,ndc,etc_rn,etccode,etcdescription,med_rn,gsn_rn
0,30000012,11714491,21562392,2126-02-14 20:22:00,Vital sign check,None,None,None,None,<NA>,None,<NA>,None,98.8,96.0,18.0,93.0,160.0,54.0,0,NaN,None,None,None,None,None,<NA>,None,None,<NA>,<NA>
1,30000012,11714491,21562392,2126-02-14 20:22:00,Enter the ED,F,WHITE,AMBULANCE,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,<NA>,None,None,<NA>,<NA>
2,30000012,11714491,21562392,2126-02-14 20:22:01,Triage in the ED,None,None,None,None,<NA>,None,<NA>,None,98.8,96.0,18.0,93.0,160.0,54.0,0,2.0,CHANGE IN MENTAL STATUS,None,None,None,None,<NA>,None,None,<NA>,<NA>
3,30000012,11714491,21562392,2126-02-14 22:21:00,Medicine reconciliation,None,None,None,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,rifaximin,066295,54868615400,1,00005844,Rifamycins and Related Derivative Antibiotics,<NA>,<NA>
4,30000012,11714491,21562392,2126-02-14 22:21:00,Medicine reconciliation,None,None,None,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,"multivitamin,tx-minerals",002510,10267073710,1,00000704,Multivitamin and Mineral Combinations,<NA>,<NA>


In [24]:
# For each stay, at each timestamp, keep only one row per activity label
# If there are multiple rows only because of different meds/diagnoses, keep the first one and let those details live in other tables.
activities = activities.sort_values(
    by=["stay_id", "timestamps", "activity"]
)
activities_dedup = activities.drop_duplicates(
    subset=["stay_id", "timestamps", "activity"],
    keep="first"
).copy()
activities_dedup.head(30)

,stay_id,subject_id,hadm_id,timestamps,activity,gender,race,arrival_transport,disposition,seq_num,icd_code,icd_version,icd_title,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint,rhythm,name,gsn,ndc,etc_rn,etccode,etcdescription,med_rn,gsn_rn
1,30000012,11714491,21562392,2126-02-14 20:22:00,Enter the ED,F,WHITE,AMBULANCE,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,<NA>,None,None,<NA>,<NA>
0,30000012,11714491,21562392,2126-02-14 20:22:00,Vital sign check,None,None,None,None,<NA>,None,<NA>,None,98.8,96.0,18.0,93.0,160.0,54.0,0,NaN,None,None,None,None,None,<NA>,None,None,<NA>,<NA>
2,30000012,11714491,21562392,2126-02-14 20:22:01,Triage in the ED,None,None,None,None,<NA>,None,<NA>,None,98.8,96.0,18.0,93.0,160.0,54.0,0,2.0,CHANGE IN MENTAL STATUS,None,None,None,None,<NA>,None,None,<NA>,<NA>
3,30000012,11714491,21562392,2126-02-14 22:21:00,Medicine reconciliation,None,None,None,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,rifaximin,066295,54868615400,1,00005844,Rifamycins and Related Derivative Antibiotics,<NA>,<NA>
10,30000012,11714491,21562392,2126-02-14 22:22:00,Medicine reconciliation,None,None,None,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,lactulose,068217,17856137801,1,00000409,Laxative - Saline and Osmotic,<NA>,<NA>
12,30000012,11714491,21562392,2126-02-14 23:43:00,Vital sign check,None,None,None,None,<NA>,None,<NA>,None,NaN,80.0,13.0,99.0,112.0,44.0,0,NaN,None,None,None,None,None,<NA>,None,None,<NA>,<NA>
13,30000012,11714491,21562392,2126-02-15 00:16:00,Medicine reconciliation,None,None,None,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,gabapentin,021413,10135064401,1,00006030,Anticonvulsant - GABA Analogs,<NA>,<NA>
14,30000012,11714491,21562392,2126-02-15 00:17:00,Medicine reconciliation,None,None,None,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,rifaximin,066295,54868615400,1,00005844,Rifamycins and Related Derivative Antibiotics,<NA>,<NA>
17,30000012,11714491,21562392,2126-02-15 00:44:00,Medicine dispensations,None,None,None,None,<NA>,None,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,Spironolactone,006817,None,<NA>,None,None,1,1
19,30000012,11714491,21562392,2126-02-15 00:50:00,Vital sign check,None,None,None,None,<NA>,None,<NA>,None,98.6,88.0,16.0,100.0,135.0,51.0,0,NaN,None,None,None,None,None,<NA>,None,None,<NA>,<NA>


In [25]:
print(f"Before dedup: {len(activities):,} rows")
print(f"After  dedup: {len(activities_dedup):,} rows")

Before dedup: 7,568,824 rows
After  dedup: 4,072,038 rows


In [26]:
# by observation a lot of med dispenstions and reconciliations happen within clusters, I want to see if I have proof of this so I can 
# collapse them with reason 
# Work on a copy so activtities df stays untouched
acts = df.copy()
acts["timestamps"] = pd.to_datetime(acts["timestamps"])

In [27]:
acts = acts[acts["activity"].notna()].copy()
acts = acts.sort_values(["stay_id", "activity", "timestamps"])
acts["delta_min"] = (
    acts.groupby(["stay_id", "activity"])["timestamps"]
        .diff()
        .dt.total_seconds() / 60
)

In [28]:
acts[["stay_id", "activity", "timestamps", "delta_min"]].head(20)

,stay_id,activity,timestamps,delta_min
21,30000012,Discharge from the ED,2126-02-15 01:59:00,NaN
22,30000012,Discharge from the ED,2126-02-15 01:59:00,0.0
1,30000012,Enter the ED,2126-02-14 20:22:00,NaN
17,30000012,Medicine dispensations,2126-02-15 00:44:00,NaN
18,30000012,Medicine dispensations,2126-02-15 00:44:00,0.0
20,30000012,Medicine dispensations,2126-02-15 01:22:00,38.0
3,30000012,Medicine reconciliation,2126-02-14 22:21:00,NaN
4,30000012,Medicine reconciliation,2126-02-14 22:21:00,0.0
5,30000012,Medicine reconciliation,2126-02-14 22:21:00,0.0
6,30000012,Medicine reconciliation,2126-02-14 22:21:00,0.0


In [29]:
# total events per activity
activity_totals = acts["activity"].value_counts()

results = {}

for threshold in [2, 5]:
    close = acts[acts["delta_min"].le(threshold)].copy()

    summary = (
        close.groupby("activity")
             .size()
             .to_frame(name=f"n_within_{threshold}min")
    )

    summary["total_events"] = activity_totals
    summary[f"prop_within_{threshold}min"] = (
        summary[f"n_within_{threshold}min"] / summary["total_events"]
    )

    results[threshold] = summary.sort_values(
        f"prop_within_{threshold}min", ascending=False
    )

    print(f"\n=== Events within {threshold} minutes of previous same-activity event ===")
    display(results[threshold].head(10))


=== Events within 2 minutes of previous same-activity event ===


,n_within_2min,total_events,prop_within_2min
activity,,,
Medicine reconciliation,2624903,2953118,0.888858
Discharge from the ED,475049,900077,0.527787
Medicine dispensations,711527,1441839,0.493486
Vital sign check,19589,1423734,0.013759



=== Events within 5 minutes of previous same-activity event ===


,n_within_5min,total_events,prop_within_5min
activity,,,
Medicine reconciliation,2631253,2953118,0.891008
Discharge from the ED,475049,900077,0.527787
Medicine dispensations,740557,1441839,0.513620
Vital sign check,44956,1423734,0.031576


In [30]:
# we are seeing a huge clustering with medicine reconciliation & medicine dispensations as predicted
# I am going to collapse these events if they are within two minutes of each other because otherwise we have the same
# problem as before but to a lesser extent 
# start from the original full event log
activities = df.copy()

# keep only the ED activities that we want in the DES timeline
activity_keep = [
    "Enter the ED",
    "Triage in the ED",
    "Discharge from the ED",
    "Vital sign check",
    "Medicine reconciliation",
    "Medicine dispensations",
]

activities = activities[activities["activity"].isin(activity_keep)].copy()
activities["timestamps"] = pd.to_datetime(activities["timestamps"])
# Quick sanity check
activities["activity"].value_counts()

activity
Medicine reconciliation    2953118
Medicine dispensations     1441839
Vital sign check           1423734
Discharge from the ED       900077
Enter the ED                425028
Triage in the ED            425028
Name: count, dtype: int64

In [31]:
activities = activities.sort_values(["stay_id", "activity", "timestamps"])

# for each stay + activity, look at the previous timestamp
activities["prev_time"] = (
    activities
    .groupby(["stay_id", "activity"])["timestamps"]
    .shift()
)

# compute minutes since previous same-activity event
activities["mins_since_prev_same_activity"] = (
    (activities["timestamps"] - activities["prev_time"])
    .dt.total_seconds() / 60
)
activities = activities.sort_values(["stay_id", "timestamps", "activity"]).reset_index(drop=True)
# keep the first event for each (stay, activity) and any later events that are > 2 minutes apart
THRESH_MIN = 2

keep_mask = (
    activities["prev_time"].isna() |
    (activities["mins_since_prev_same_activity"] > THRESH_MIN)
)

ed_activities = activities.loc[keep_mask].copy()
ed_activities = ed_activities.drop(columns=["prev_time"])
# drop all irrelevant cols
cols_keep = [
    "stay_id",
    "subject_id",
    "hadm_id",
    "timestamps",
    "activity",
    "acuity",
]
cols_keep = [c for c in cols_keep if c in ed_activities.columns]
ed_activities = ed_activities[cols_keep].reset_index(drop=True)

In [32]:
before_counts = activities["activity"].value_counts().rename("before")
after_counts = ed_activities["activity"].value_counts().rename("after")

summary_counts = (
    pd.concat([before_counts, after_counts], axis=1)
    .assign(
        reduced=lambda x: x["before"] - x["after"],
        frac_reduced=lambda x: x["reduced"] / x["before"]
    )
)

summary_counts

,before,after,reduced,frac_reduced
activity,,,,
Medicine reconciliation,2953118,328215,2624903,0.888858
Medicine dispensations,1441839,730312,711527,0.493486
Vital sign check,1423734,1404145,19589,0.013759
Discharge from the ED,900077,425028,475049,0.527787
Enter the ED,425028,425028,0,0.000000
Triage in the ED,425028,425028,0,0.000000


In [33]:
# this looks MUCH better after applying the two minute clsutering threshold
ed_activities.head(20)

,stay_id,subject_id,hadm_id,timestamps,activity,acuity
0,30000012,11714491,21562392,2126-02-14 20:22:00,Enter the ED,NaN
1,30000012,11714491,21562392,2126-02-14 20:22:00,Vital sign check,NaN
2,30000012,11714491,21562392,2126-02-14 20:22:01,Triage in the ED,2.0
3,30000012,11714491,21562392,2126-02-14 22:21:00,Medicine reconciliation,NaN
4,30000012,11714491,21562392,2126-02-14 23:43:00,Vital sign check,NaN
5,30000012,11714491,21562392,2126-02-15 00:16:00,Medicine reconciliation,NaN
6,30000012,11714491,21562392,2126-02-15 00:44:00,Medicine dispensations,NaN
7,30000012,11714491,21562392,2126-02-15 00:50:00,Vital sign check,NaN
8,30000012,11714491,21562392,2126-02-15 01:22:00,Medicine dispensations,NaN
9,30000012,11714491,21562392,2126-02-15 01:59:00,Discharge from the ED,NaN


In [43]:
ed_activities["activity_seq"] = ed_activities.groupby("stay_id").cumcount() + 1
ed_activities.head(25)

,stay_id,subject_id,hadm_id,timestamps,activity,acuity,activity_seq
0,30000012,11714491,21562392,2126-02-14 20:22:00,Enter the ED,NaN,1
1,30000012,11714491,21562392,2126-02-14 20:22:00,Vital sign check,NaN,2
2,30000012,11714491,21562392,2126-02-14 20:22:01,Triage in the ED,2.0,3
3,30000012,11714491,21562392,2126-02-14 22:21:00,Medicine reconciliation,NaN,4
4,30000012,11714491,21562392,2126-02-14 23:43:00,Vital sign check,NaN,5
5,30000012,11714491,21562392,2126-02-15 00:16:00,Medicine reconciliation,NaN,6
6,30000012,11714491,21562392,2126-02-15 00:44:00,Medicine dispensations,NaN,7
7,30000012,11714491,21562392,2126-02-15 00:50:00,Vital sign check,NaN,8
8,30000012,11714491,21562392,2126-02-15 01:22:00,Medicine dispensations,NaN,9
9,30000012,11714491,21562392,2126-02-15 01:59:00,Discharge from the ED,NaN,10


In [44]:
# time for sanity checks!
# check if events happen AFTER they should
# for exmaple, discharge before triage is bad, but missing vitals would be okay
first_events = (
    ed_activities.pivot_table(
        index="stay_id",
        columns="activity",
        values="timestamps",
        aggfunc="min"
    )
)
critical_order_pairs = [
    ("Enter the ED", "Triage in the ED"),
    ("Enter the ED", "Discharge from the ED"),
    ("Triage in the ED", "Discharge from the ED"),
]

for earlier, later in critical_order_pairs:
    violations = first_events[
        (first_events[earlier].notna()) & 
        (first_events[later].notna()) &
        (first_events[later] < first_events[earlier])
    ]
    print(f"{later} before {earlier}: {len(violations)} stays")

Triage in the ED before Enter the ED: 0 stays
Discharge from the ED before Enter the ED: 0 stays
Discharge from the ED before Triage in the ED: 0 stays


In [45]:
# check for stays with only entry and no other activities and inter-activity timing distributions to see if it looks normal 
activity_counts = ed_activities.groupby("stay_id")["activity"].count()
single_activity_stays = activity_counts[activity_counts == 1]
print(f"Stays with only 1 activity: {len(single_activity_stays)}")
ed_activities = ed_activities.sort_values(["stay_id", "timestamps"])
ed_activities["time_to_next_activity_min"] = (
    ed_activities.groupby("stay_id")["timestamps"]
    .diff(-1)
    .abs()
    .dt.total_seconds() / 60
)

print("\nInter-activity timing (minutes):")
print(ed_activities.groupby("activity")["time_to_next_activity_min"].describe())

# Check for same-second timestamps (data artifact)
same_second_events = ed_activities.duplicated(subset=["stay_id", "timestamps"], keep=False)
print(f"\nEvents with same-second timestamps: {same_second_events.sum()}")

# Verify all stays in activities exist in stays table
orphan_activities = ~ed_activities["stay_id"].isin(ed_stays["stay_id"])
print(f"Activity records for non-existent stays: {orphan_activities.sum()}")
ed_activities = ed_activities.drop(columns=["time_to_next_activity_min"])

Stays with only 1 activity: 0

Inter-activity timing (minutes):
                             count       mean         std       min        25%        50%        75%            max
activity                                                                                                           
Discharge from the ED          0.0        NaN         NaN       NaN        NaN        NaN        NaN            NaN
Enter the ED              425028.0   0.013588    0.006468  0.000000   0.016667   0.016667   0.016667       0.016667
Medicine dispensations    730312.0  55.320850  158.346261  0.000000   9.000000  31.000000  77.000000  117989.000000
Medicine reconciliation   328215.0  65.560318   67.893405  0.000000  13.000000  46.000000  96.000000    1271.966667
Triage in the ED          425028.0  56.933102   76.175648  0.983333   1.983333  25.983333  83.983333    1890.616667
Vital sign check         1404145.0  69.376666   99.338009  0.016667  12.000000  44.000000  98.000000   40201.000000

Events 

In [40]:
required = ["Enter the ED", "Triage in the ED", "Discharge from the ED"]
missing_stays = {
    evt: missing_evt[missing_evt].index.tolist()
    for evt, missing_evt in 
    {
        evt: ed_activities.groupby("stay_id")["activity"].apply(lambda x: evt not in x.values)
        for evt in required
    }.items()
}

missing_stays
# no missing stays, this is good

{'Enter the ED': [], 'Triage in the ED': [], 'Discharge from the ED': []}

In [46]:
# verify every stay has both enter and discharge
has_enter = ed_activities.groupby("stay_id")["activity"].apply(
    lambda x: "Enter the ED" in x.values
)
has_discharge = ed_activities.groupby("stay_id")["activity"].apply(
    lambda x: "Discharge from the ED" in x.values
)

print(f"Stays missing 'Enter the ED': {(~has_enter).sum()}")
print(f"Stays missing 'Discharge from the ED': {(~has_discharge).sum()}")

# activity counts per stay look reasonable
print("\nActivities per stay distribution:")
print(activity_counts.describe())

Stays missing 'Enter the ED': 0
Stays missing 'Discharge from the ED': 0

Activities per stay distribution:
count    425028.000000
mean          8.794141
std           4.550270
min           3.000000
25%           6.000000
50%           8.000000
75%          10.000000
max          89.000000
Name: activity, dtype: float64


## Activities Table Data Quality Validation Summary

**Completeness**: All 425,028 stays have both "Enter the ED" and "Discharge from the ED"  
**Sequencing**: No critical ordering violations (discharge before arrival, etc.)  
**Activity counts**: Mean 8.8 activities per stay (median=8), range 3-89 - reasonable distribution  
**Orphan records**: 0 activities for non-existent stays  
**Temporal clustering**: 2-minute threshold successfully reduced redundant events  

**Decision**: Table is clean and ready for DES parameterization.

In [ ]:
# save this row for creating the csv table for activities 

In [47]:
ed_activities.head(25)

,stay_id,subject_id,hadm_id,timestamps,activity,acuity,activity_seq
0,30000012,11714491,21562392,2126-02-14 20:22:00,Enter the ED,NaN,1
1,30000012,11714491,21562392,2126-02-14 20:22:00,Vital sign check,NaN,2
2,30000012,11714491,21562392,2126-02-14 20:22:01,Triage in the ED,2.0,3
3,30000012,11714491,21562392,2126-02-14 22:21:00,Medicine reconciliation,NaN,4
4,30000012,11714491,21562392,2126-02-14 23:43:00,Vital sign check,NaN,5
5,30000012,11714491,21562392,2126-02-15 00:16:00,Medicine reconciliation,NaN,6
6,30000012,11714491,21562392,2126-02-15 00:44:00,Medicine dispensations,NaN,7
7,30000012,11714491,21562392,2126-02-15 00:50:00,Vital sign check,NaN,8
8,30000012,11714491,21562392,2126-02-15 01:22:00,Medicine dispensations,NaN,9
9,30000012,11714491,21562392,2126-02-15 01:59:00,Discharge from the ED,NaN,10
